In [1]:
pip install fastembed


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
from fastembed import TextEmbedding

In [4]:
embed_model = TextEmbedding(model_name="jinaai/jina-embeddings-v2-small-en")

In [5]:
query_text = "I just discovered the course. Can I join now?"
query_embedding = next(embed_model.embed([query_text]))

In [8]:
import numpy as np

query_embedding = np.array(query_embedding)
min_value = np.min(query_embedding)

print(f"Minimum value in the embedding: {min_value}")


Minimum value in the embedding: -0.11726373885183883


In [10]:
import numpy as np
print(np.linalg.norm(query_embedding))

1.0


In [12]:
cos_sim = query_embedding.dot(query_embedding)
print("Cosine similarity with itself:", cos_sim)

Cosine similarity with itself: 1.0000000000000002


In [13]:
doc = 'Can I still join the course after the start date?'
d_vec = np.array(next(embed_model.embed([doc])))

In [15]:
cos_sim = np.dot(query_embedding, d_vec)

print(f"Cosine similarity between query and document: {cos_sim:.4f}")

Cosine similarity between query and document: 0.9009


In [16]:
documents = [{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
  'section': 'General course-related questions',
  'question': 'How can we contribute to the course?',
  'course': 'data-engineering-zoomcamp'}]

In [17]:
doc_texts = [doc["text"] for doc in documents]
doc_embeddings = list(embed_model.embed(doc_texts))

In [19]:
V = np.vstack(doc_embeddings)  # Shape: (5, 512)
similarities = V.dot(query_embedding)  # Shape: (5,)
most_similar_idx = int(np.argmax(similarities))
print("Most similar document index:", most_similar_idx)

Most similar document index: 1


In [24]:
full_texts = [doc["question"] + " " + doc["text"] for doc in documents]

In [25]:
full_embeddings = list(embed_model.embed(full_texts))
V = np.vstack(full_embeddings)  # Shape: (5, 512)

# Compute cosine similarity
similarities = V.dot(query_embedding)

# Find the most similar
most_similar_idx = int(np.argmax(similarities))
print("Most similar document index (Q4):", most_similar_idx)

Most similar document index (Q4): 0


In [27]:
from fastembed.embedding import TextEmbedding
import numpy as np

# Initialize the smaller model
embed_model = TextEmbedding(model_name="BAAI/bge-small-en")

# Embed a query
query = "I just discovered the course. Can I join now?"
query_vec = np.array(next(embed_model.embed([query])))

# Check vector size
print("Embedding shape:", query_vec.shape)  # Should be (384,)


Embedding shape: (384,)


In [ ]:
import requests
import numpy as np
from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams, Distance, PointStruct
from fastembed import TextEmbedding

# Load and filter documents
docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
documents_raw = requests.get(docs_url).json()

documents = []
for course in documents_raw:
    if course['course'] == 'machine-learning-zoomcamp':
        for doc in course['documents']:
            doc['course'] = course['course']
            documents.append(doc)

# Prepare full text (question + text)
full_texts = [doc["question"] + " " + doc["text"] for doc in documents]

# Use BAAI/bge-small-en (dim = 384)
embed_model = TextEmbedding(model_name="BAAI/bge-small-en")
embeddings = list(embed_model.embed(full_texts))

# Init Qdrant in-memory
client = QdrantClient(host="localhost", port=6333)
collection_name = "ml_faq"
client.recreate_collection(
    collection_name=collection_name,
    vectors_config=VectorParams(size=384, distance=Distance.COSINE)
)

# Insert documents into Qdrant
points = [
    PointStruct(id=i, vector=vec, payload={"text": text})
    for i, (vec, text) in enumerate(zip(embeddings, full_texts))
]
client.upsert(collection_name=collection_name, points=points)

# Query from Q1
query = "I just discovered the course. Can I join now?"
query_vec = np.array(next(embed_model.embed([query])))

# Perform search
results = client.search(
    collection_name=collection_name,
    query_vector=query_vec,
    limit=1,
    with_payload=True
)

# Show top score
top_score = results[0].score
print(f"Top similarity score: {top_score:.2f}")

